#### Importações necessárias
***

In [90]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
import dotenv
dotenv.load_dotenv('../config/.env')


True

#### Extraindo e mostrando os dados dos datasets
***

In [89]:
PATH = '../datasets/dados_clinicos/dados_clinicos.csv'
df_dados_clinicos = pd.read_csv(PATH, sep=';', encoding='latin1')

In [88]:
PATH = '../datasets/dados_estado_regiao/estado_regiao.csv'
df_estado_regiao = pd.read_csv(PATH, sep=';', encoding='latin1')

In [87]:
PATH = '../datasets/dados_pacientes/dados_pacientes.csv'
df_dados_pacientes = pd.read_csv(PATH, sep=';', encoding='latin1')

In [86]:
df_dados_pacientes.head()

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario
0,1,39,Servidor Público,Ensino Médio Completo,2,Solteiro,Branco,2.0,4754
1,2,50,Autônomo,Superior Incompleto,24,Casado,Branco,1.0,3923
2,3,38,Funcionário Setor Privado,Ensino Médio Incompleto,4,Divorciado,Branco,0.0,1100
3,4,53,Funcionário Setor Privado,Ensino Médio Incompleto,24,Casado,Negro,1.0,1100
4,5,28,Funcionário Setor Privado,Ensino Médio Completo,15,Casado,Negro,0.0,3430


In [85]:
df_dados_clinicos.head()

,id_cliente,peso,colesterol,genero
0,1,152.7,203.9,Masculino
1,2,184.8,233.8,Masculino
2,3,152.1,210.4,Masculino
3,4,139.8,196.8,Masculino
4,5,182.4,215.8,Feminino


In [84]:
df_estado_regiao.head()

,id_estado,sigla,estado,regiao,pais
0,1,AC,Acre,Norte,Brasil
1,2,AL,Alagoas,Nordeste,Brasil
2,3,AP,Amapá,Norte,Brasil
3,4,AM,Amazonas,Norte,Brasil
4,5,BA,Bahia,Nordeste,Brasil


#### Verificando e corrigindo os dados duplicados
***

In [82]:
def verifica_dados_duplicados(nome_dataframe, dataframe):
    qtd = dataframe.duplicated().sum()

    print(f'O dataframe {nome_dataframe} possui {qtd} dados duplicados')

In [93]:
verifica_dados_duplicados('dados_clinicos', df_dados_clinicos)
verifica_dados_duplicados('dados_pacientes', df_dados_pacientes)
verifica_dados_duplicados('dados_estado_regiao', df_estado_regiao)


O dataframe dados_clinicos possui 301 dados duplicados
O dataframe dados_pacientes possui 85 dados duplicados
O dataframe dados_estado_regiao possui 9 dados duplicados


In [94]:
df_dados_pacientes = df_dados_pacientes.drop_duplicates()
verifica_dados_duplicados('df_dados_pacientes', df_dados_pacientes)

O dataframe df_dados_pacientes possui 0 dados duplicados


In [95]:
df_dados_clinicos = df_dados_clinicos.drop_duplicates()
verifica_dados_duplicados('df_dados_clinicos', df_dados_clinicos)

O dataframe df_dados_clinicos possui 0 dados duplicados


In [96]:
df_estado_regiao = df_estado_regiao.drop_duplicates()
verifica_dados_duplicados('df_estado_regiao', df_estado_regiao)

O dataframe df_estado_regiao possui 0 dados duplicados


In [97]:
lista_dataframes = [
    ('dados_clinicos', df_dados_clinicos),
    ('dados_pacientes', df_dados_pacientes),
    ('dados_estado_regiao', df_estado_regiao)
]

for nome_dataframe, dataframe in lista_dataframes:
           
    print('--' * 50)
    verifica_dados_duplicados(nome_dataframe, dataframe)
print('--' * 50)

----------------------------------------------------------------------------------------------------
O dataframe dados_clinicos possui 0 dados duplicados
----------------------------------------------------------------------------------------------------
O dataframe dados_pacientes possui 0 dados duplicados
----------------------------------------------------------------------------------------------------
O dataframe dados_estado_regiao possui 0 dados duplicados
----------------------------------------------------------------------------------------------------


#### Verificando e corrigindo os dados ausentes dos dados_pacientes
***

In [98]:
def verifica_dados_ausentes(nome_dataframe, dataframe):
    dados_ausentes = dataframe.isna().sum()
    colunas_com_ausentes = dados_ausentes[dados_ausentes > 0]
    
    print('--' * 50)
    
    if not colunas_com_ausentes.empty:
        print(f'O dataframe {nome_dataframe} possui dados ausentes.\nLista de colunas:')
        return print(colunas_com_ausentes)
    else:
        print(f'O dataframe {nome_dataframe} não possui dados ausentes.')

In [101]:
verifica_dados_ausentes('df_dados_pacientes', df_dados_pacientes)

----------------------------------------------------------------------------------------------------
O dataframe df_dados_pacientes possui dados ausentes.
Lista de colunas:
classe_trabalho    387
qtde_filhos         10
dtype: int64


In [103]:
moda_qtde_filhos = df_dados_pacientes['qtde_filhos'].median()
print(moda_qtde_filhos)
df_dados_pacientes['qtde_filhos'] = df_dados_pacientes['qtde_filhos'].fillna(value=moda_qtde_filhos)

2.0


In [105]:
moda_classe_trabalho = df_dados_pacientes['classe_trabalho'].mode()[0]
print(moda_classe_trabalho)
df_dados_pacientes['classe_trabalho'] = df_dados_pacientes['classe_trabalho'].fillna(value=moda_classe_trabalho)

Funcionário Setor Privado


In [106]:
verifica_dados_ausentes('df_dados_pacientes', df_dados_pacientes)

----------------------------------------------------------------------------------------------------
O dataframe df_dados_pacientes não possui dados ausentes.


#### Verificando e corrigindo os dados ausentes dos dados_clínicos
***

In [107]:
verifica_dados_ausentes('df_dados_clinicos', df_dados_clinicos)

----------------------------------------------------------------------------------------------------
O dataframe df_dados_clinicos possui dados ausentes.
Lista de colunas:
peso      13
genero    10
dtype: int64


In [108]:
filtro = df_dados_clinicos['peso'].isna()
index_ausentes_peso = df_dados_clinicos[filtro].index
df_dados_clinicos.loc[index_ausentes_peso]

,id_cliente,peso,colesterol,genero
10224,9924,NaN,201.1,Feminino
10225,9925,NaN,225.2,Masculino
10230,9930,NaN,134.9,Feminino
10231,9931,NaN,237.5,Feminino
10243,9943,NaN,219.3,Feminino
10244,9944,NaN,246.2,NaN
10248,9948,NaN,184.0,Masculino
10251,9951,NaN,182.5,Feminino
10262,9962,NaN,115.9,Masculino
10270,9970,NaN,173.1,Masculino


In [109]:
mediana_peso = df_dados_clinicos['peso'].median()
print(mediana_peso)
df_dados_clinicos['peso'] = df_dados_clinicos['peso'].fillna(value=mediana_peso)

124.6


In [110]:
filtro = df_dados_clinicos['genero'].isna()
index_ausentes_genero = df_dados_clinicos[filtro].index
df_dados_clinicos.loc[index_ausentes_genero]

,id_cliente,peso,colesterol,genero
10227,9927,177.6,220.3,NaN
10228,9928,149.3,199.3,NaN
10233,9933,65.3,132.8,NaN
10234,9934,180.6,214.1,NaN
10237,9937,103.6,160.8,NaN
10240,9940,81.6,165.8,NaN
10241,9941,154.9,213.6,NaN
10242,9942,96.3,163.0,NaN
10244,9944,124.6,246.2,NaN
10245,9945,51.4,130.5,NaN


In [111]:
moda_genero = df_dados_clinicos['genero'].mode()[0]
print(moda_genero)
df_dados_clinicos['genero'] = df_dados_clinicos['genero'].fillna(value=moda_genero)

Feminino


In [112]:
df_dados_clinicos.loc[index_ausentes_genero]

,id_cliente,peso,colesterol,genero
10227,9927,177.6,220.3,Feminino
10228,9928,149.3,199.3,Feminino
10233,9933,65.3,132.8,Feminino
10234,9934,180.6,214.1,Feminino
10237,9937,103.6,160.8,Feminino
10240,9940,81.6,165.8,Feminino
10241,9941,154.9,213.6,Feminino
10242,9942,96.3,163.0,Feminino
10244,9944,124.6,246.2,Feminino
10245,9945,51.4,130.5,Feminino


In [113]:
verifica_dados_ausentes('df_dados_clinicos', df_dados_clinicos)

----------------------------------------------------------------------------------------------------
O dataframe df_dados_clinicos não possui dados ausentes.


#### Verificando e corrigindo os dados ausentes dos dados_estado_regiao
***

In [114]:
verifica_dados_ausentes('df_estado_regiao', df_estado_regiao)

----------------------------------------------------------------------------------------------------
O dataframe df_estado_regiao não possui dados ausentes.


In [115]:
df_estado_regiao.shape

(27, 5)

In [116]:
df_estado_regiao.head(27)

,id_estado,sigla,estado,regiao,pais
0,1,AC,Acre,Norte,Brasil
1,2,AL,Alagoas,Nordeste,Brasil
2,3,AP,Amapá,Norte,Brasil
3,4,AM,Amazonas,Norte,Brasil
4,5,BA,Bahia,Nordeste,Brasil
5,6,CE,Ceará,Nordeste,Brasil
6,7,DF,Distrito Federal,Centro-Oeste,Brasil
7,8,ES,Espírito Santo,Sudeste,Brasil
8,9,GO,Goiás,Centro-Oeste,Brasil
9,10,MA,Maranhão,Nordeste,Brasil


In [117]:
print(df_estado_regiao.dtypes)
print('--' * 50)
print(df_dados_clinicos.dtypes)
print('--' * 50)
print(df_dados_pacientes.dtypes)

id_estado     int64
sigla        object
estado       object
regiao       object
pais         object
dtype: object
----------------------------------------------------------------------------------------------------
id_cliente      int64
peso          float64
colesterol    float64
genero         object
dtype: object
----------------------------------------------------------------------------------------------------
id_cliente           int64
idade                int64
classe_trabalho     object
escolaridade        object
id_estado            int64
estado_civil        object
raca                object
qtde_filhos        float64
salario              int64
dtype: object


#### Corrigindo os tipos de qtde_filhos e salario

In [119]:
df_dados_pacientes['salario'] = df_dados_pacientes['salario'].astype(float).round(2)


In [118]:
df_dados_pacientes['qtde_filhos'] = df_dados_pacientes['qtde_filhos'].astype(int)

In [120]:
df_dados_pacientes.head()

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario
0,1,39,Servidor Público,Ensino Médio Completo,2,Solteiro,Branco,2,4754.0
1,2,50,Autônomo,Superior Incompleto,24,Casado,Branco,1,3923.0
2,3,38,Funcionário Setor Privado,Ensino Médio Incompleto,4,Divorciado,Branco,0,1100.0
3,4,53,Funcionário Setor Privado,Ensino Médio Incompleto,24,Casado,Negro,1,1100.0
4,5,28,Funcionário Setor Privado,Ensino Médio Completo,15,Casado,Negro,0,3430.0


#### Corrigindo os floats para 2 casas decimais
***

In [121]:
df_dados_clinicos['peso'] = df_dados_clinicos['peso'].round(2)


In [122]:
df_dados_clinicos['colesterol'] = df_dados_clinicos['colesterol'].round(2)

In [124]:
df_dados_clinicos.head()

,id_cliente,peso,colesterol,genero
0,1,152.7,203.9,Masculino
1,2,184.8,233.8,Masculino
2,3,152.1,210.4,Masculino
3,4,139.8,196.8,Masculino
4,5,182.4,215.8,Feminino


### Conexão com o Banco de Dados
***

In [151]:
def conectar_banco_mysql():
    try:
        user = os.environ['user_db']
        password = os.environ['password_db']
        host = os.environ['host']

        conexao = f'mysql://{user}:{password}@{host}'
        engine = create_engine(conexao)
        conn = engine.connect()
        print('Conexão Realizada com Sucesso!')
        return conn
    except Exception as e:
        print(f'Não foi possível realizar a conexão. Erro: {e}')

In [152]:
conn = conectar_banco_mysql()

Conexão Realizada com Sucesso!


#### Criação do Banco de Dados
***

In [153]:
try:
    database = 'enunciado_clinica'
    query = f'create schema if not exists {database}'
    conn.execute(text(query))
    print('Banco de dados criado com sucesso!')
    conn.commit()
except Exception as e:
    print(f'Não foi possível criar o banco de dados. Erro: {e}')

Banco de dados criado com sucesso!


#### Executa o Script de Criação das Tabelas e Seus Relacionamentos

In [154]:
with open('../projetos/script_create_DB.sql') as file:
    query = file.read()

try:
    conn.execute(text(query))
    conn.commit()
    print('Tabelas criadas com sucesso!')
except Exception as e:
    print(f'Não foi possível criar as tabelas do banco de dados. Erro: {e}')

Tabelas criadas com sucesso!


#### Mesclando os dataframes resultantes dos datasets
***

In [128]:
df_primeira_mesclagem = pd.merge(left=df_dados_pacientes, right=df_dados_clinicos, left_on='id_cliente', right_on='id_cliente', how='inner')

In [129]:
df_completo = pd.merge(left=df_primeira_mesclagem, right=df_estado_regiao, left_on='id_estado', right_on='id_estado', how='inner')

#### Mostrando o resultado da mesclagem
***

In [130]:
df_completo.columns

Index(['id_cliente', 'idade', 'classe_trabalho', 'escolaridade', 'id_estado',
       'estado_civil', 'raca', 'qtde_filhos', 'salario', 'peso', 'colesterol',
       'genero', 'sigla', 'estado', 'regiao', 'pais'],
      dtype='object')

In [135]:
cols_pacientes = ['id_cliente', 'idade', 'classe_trabalho', 'escolaridade', 'id_estado', 'estado_civil', 'raca', 'qtde_filhos', 'salario']
df_dados_pacientes = df_completo[cols_pacientes].drop_duplicates()

In [138]:
cols_dados_clinicos = ['id_cliente', 'peso', 'colesterol', 'genero']
df_dados_clinicos = df_completo[cols_dados_clinicos].drop_duplicates()

In [139]:
cols_dados_estados = ['id_estado', 'sigla', 'estado', 'regiao', 'pais']
df_estado_regiao = df_completo[cols_dados_estados].drop_duplicates()

In [181]:
moda_qtde_filhos = df_completo['qtde_filhos'].median()
print(moda_qtde_filhos)
df_completo['qtde_filhos'] = df_completo['qtde_filhos'].fillna(value=moda_qtde_filhos)

2.0


In [182]:
df_completo.head()

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario,peso,colesterol,genero,sigla,estado,regiao,pais
0,1,39,Servidor Público,Ensino Médio Completo,2,Solteiro,Branco,2,4754.0,152.7,203.9,Masculino,AL,Alagoas,Nordeste,Brasil
1,2,50,Autônomo,Superior Incompleto,24,Casado,Branco,1,3923.0,184.8,233.8,Masculino,SC,Santa Catarina,Sul,Brasil
2,3,38,Funcionário Setor Privado,Ensino Médio Incompleto,4,Divorciado,Branco,0,1100.0,152.1,210.4,Masculino,AM,Amazonas,Norte,Brasil
3,4,53,Funcionário Setor Privado,Ensino Médio Incompleto,24,Casado,Negro,1,1100.0,139.8,196.8,Masculino,SC,Santa Catarina,Sul,Brasil
4,5,28,Funcionário Setor Privado,Ensino Médio Completo,15,Casado,Negro,0,3430.0,182.4,215.8,Feminino,PB,Paraíba,Nordeste,Brasil


In [144]:
df_completo.shape

(7999, 16)

In [145]:
lista_classe_trabalho = list(df_completo['classe_trabalho'].unique())
lista_classe_trabalho

['Servidor Público',
 'Autônomo',
 'Funcionário Setor Privado',
 'Funcionário Público',
 'Empresário',
 'Estagiário',
 'Desempregado',
 'MEI',
 'Aposentado',
 'Menor Aprendiz']

In [146]:
lista_estado_civil = list(df_completo['estado_civil'].unique())
lista_estado_civil

['Solteiro', 'Casado', 'Divorciado', 'União Estável', 'Separado', 'Viúvo']

In [147]:
lista_raca = list(df_completo['raca'].unique())
lista_raca

['Branco', 'Negro', 'Pardo', 'Indígena', 'Amarelo']

In [148]:
lista_escolaridade = list(df_completo['escolaridade'].unique())
lista_escolaridade

['Ensino Médio Completo',
 'Superior Incompleto',
 'Ensino Médio Incompleto',
 'Mestrado',
 'Ensino Fundamental Completo',
 'Superior Completo',
 'Pós-Graduado',
 'Ensino Fundamental Incompleto',
 'Doutorado',
 'Analfabeto']

In [149]:
for registro in df_estado_regiao.itertuples():
    print(registro)

Pandas(Index=0, id_estado=2, sigla='AL', estado='Alagoas', regiao='Nordeste', pais='Brasil')
Pandas(Index=1, id_estado=24, sigla='SC', estado='Santa Catarina', regiao='Sul', pais='Brasil')
Pandas(Index=2, id_estado=4, sigla='AM', estado='Amazonas', regiao='Norte', pais='Brasil')
Pandas(Index=4, id_estado=15, sigla='PB', estado='Paraíba', regiao='Nordeste', pais='Brasil')
Pandas(Index=6, id_estado=23, sigla='RR', estado='Roraima', regiao='Norte', pais='Brasil')
Pandas(Index=7, id_estado=13, sigla='MG', estado='Minas Gerais', regiao='Sudeste', pais='Brasil')
Pandas(Index=8, id_estado=17, sigla='PE', estado='Pernambuco', regiao='Nordeste', pais='Brasil')
Pandas(Index=9, id_estado=16, sigla='PR', estado='Paraná', regiao='Sul', pais='Brasil')
Pandas(Index=11, id_estado=12, sigla='MS', estado='Mato Grosso do Sul', regiao='Centro-Oeste', pais='Brasil')
Pandas(Index=12, id_estado=22, sigla='RO', estado='Rondônia', regiao='Norte', pais='Brasil')
Pandas(Index=13, id_estado=3, sigla='AP', estado=

In [156]:
for registro in df_estado_regiao.itertuples():
    cod_estado = registro.id_estado
    estado = registro.estado
    sigla_estado = registro.sigla
    regiao = registro.regiao
    pais = registro.pais

    try:
        query = f'''
                insert into tb_estado (cod_estado, estado, sigla, regiao, pais)
                values({cod_estado}, "{estado}", "{sigla_estado}", "{regiao}", "{pais}")
    '''
        conn.execute(text(query))
        conn.commit()
        print(f'Registro inserido com sucesso. Estado: {estado}')

    except Exception as e:
        print(f'Não foi possivel inserir o registro: {estado}. Erro: {e}')

Registro inserido com sucesso. Estado: Alagoas
Registro inserido com sucesso. Estado: Santa Catarina
Registro inserido com sucesso. Estado: Amazonas
Registro inserido com sucesso. Estado: Paraíba
Registro inserido com sucesso. Estado: Roraima
Registro inserido com sucesso. Estado: Minas Gerais
Registro inserido com sucesso. Estado: Pernambuco
Registro inserido com sucesso. Estado: Paraná
Registro inserido com sucesso. Estado: Mato Grosso do Sul
Registro inserido com sucesso. Estado: Rondônia
Registro inserido com sucesso. Estado: Amapá
Registro inserido com sucesso. Estado: Pará
Registro inserido com sucesso. Estado: Goiás
Registro inserido com sucesso. Estado: Espírito Santo
Registro inserido com sucesso. Estado: Tocantins
Registro inserido com sucesso. Estado: Distrito Federal
Registro inserido com sucesso. Estado: São Paulo
Registro inserido com sucesso. Estado: Rio de Janeiro
Registro inserido com sucesso. Estado: Rio Grande do Sul
Registro inserido com sucesso. Estado: Ceará
Regis

#### Inserindo Registros no Banco de Dados
***

In [158]:
def insert_dados_bancos(lista_dados, tabela):
    coluna_tabela = tabela[3:]
    
    for registro in lista_dados:
        try:
           
            query = f'''
            INSERT INTO {tabela} ({coluna_tabela})
                        VALUES("{registro}")            
            '''
            conn.execute(text(query))
            conn.commit()
            
            print(f'Registro inserido com sucesso: {registro}')
        except Exception as e:
            print(f'Não foi possível inserir o registro: {registro}. Erro: {e}')

In [159]:
tabela_banco = 'tb_classe_trabalho'
insert_dados_bancos(lista_classe_trabalho, tabela_banco)

Registro inserido com sucesso: Servidor Público
Registro inserido com sucesso: Autônomo
Registro inserido com sucesso: Funcionário Setor Privado
Registro inserido com sucesso: Funcionário Público
Registro inserido com sucesso: Empresário
Registro inserido com sucesso: Estagiário
Registro inserido com sucesso: Desempregado
Registro inserido com sucesso: MEI
Registro inserido com sucesso: Aposentado
Registro inserido com sucesso: Menor Aprendiz


In [160]:
tabela_banco = 'tb_estado_civil'
insert_dados_bancos(lista_estado_civil, tabela_banco)

Registro inserido com sucesso: Solteiro
Registro inserido com sucesso: Casado
Registro inserido com sucesso: Divorciado
Registro inserido com sucesso: União Estável
Registro inserido com sucesso: Separado
Registro inserido com sucesso: Viúvo


In [161]:
tabela_banco = 'tb_raca'
insert_dados_bancos(lista_raca, tabela_banco)

Registro inserido com sucesso: Branco
Registro inserido com sucesso: Negro
Registro inserido com sucesso: Pardo
Registro inserido com sucesso: Indígena
Registro inserido com sucesso: Amarelo


In [184]:
tabela_banco = 'tb_escolaridade'
insert_dados_bancos(lista_escolaridade, tabela_banco)

Registro inserido com sucesso: Ensino Médio Completo
Registro inserido com sucesso: Superior Incompleto
Registro inserido com sucesso: Ensino Médio Incompleto
Registro inserido com sucesso: Mestrado
Registro inserido com sucesso: Ensino Fundamental Completo
Registro inserido com sucesso: Superior Completo
Registro inserido com sucesso: Pós-Graduado
Registro inserido com sucesso: Ensino Fundamental Incompleto
Registro inserido com sucesso: Doutorado
Registro inserido com sucesso: Analfabeto


In [163]:
df_completo.shape

(7999, 16)

In [178]:
df_completo.head()

,id_cliente,idade,classe_trabalho,escolaridade,id_estado,estado_civil,raca,qtde_filhos,salario,peso,colesterol,genero,sigla,estado,regiao,pais
0,1,39,Servidor Público,Ensino Médio Completo,2,Solteiro,Branco,2,4754.0,152.7,203.9,Masculino,AL,Alagoas,Nordeste,Brasil
1,2,50,Autônomo,Superior Incompleto,24,Casado,Branco,1,3923.0,184.8,233.8,Masculino,SC,Santa Catarina,Sul,Brasil
2,3,38,Funcionário Setor Privado,Ensino Médio Incompleto,4,Divorciado,Branco,0,1100.0,152.1,210.4,Masculino,AM,Amazonas,Norte,Brasil
3,4,53,Funcionário Setor Privado,Ensino Médio Incompleto,24,Casado,Negro,1,1100.0,139.8,196.8,Masculino,SC,Santa Catarina,Sul,Brasil
4,5,28,Funcionário Setor Privado,Ensino Médio Completo,15,Casado,Negro,0,3430.0,182.4,215.8,Feminino,PB,Paraíba,Nordeste,Brasil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,36,Funcionário Setor Privado,Ensino Médio Completo,12,Divorciado,Branco,2,3466.0,96.6,150.0,Masculino,MS,Mato Grosso do Sul,Centro-Oeste,Brasil
196,197,40,Funcionário Público,Mestrado,17,Solteiro,Amarelo,4,7352.0,125.8,168.8,Masculino,PE,Pernambuco,Nordeste,Brasil
197,198,35,Funcionário Setor Privado,Mestrado,26,Casado,Branco,4,8748.0,126.1,177.2,Feminino,SE,Sergipe,Nordeste,Brasil
198,199,24,Autônomo,Ensino Médio Incompleto,2,Solteiro,Branco,2,1100.0,128.5,170.4,Masculino,AL,Alagoas,Nordeste,Brasil


In [183]:
try:
    df_completo.to_sql('stg_enunciado',
              schema='enunciado_clinica',
              index=False,
              con=conn
        
    )
    print(f'Tabela Stage criada com sucesso.')

except Exception as e:
    print(f'Não foi possivel inserir o registro: {estado}. Erro: {e}')

Não foi possivel inserir o registro: Rio Grande do Norte. Erro: Table 'stg_enunciado' already exists.


In [173]:
conn.commit()

In [174]:
conn.execute(text('use enunciado_clinica'))

In [185]:
query = '''
insert into tb_paciente(
    cod_paciente,
    genero,
    idade,
    qtd_filho,
    salario,
    peso,
    colesterol,
    cod_estado_civil,
    cod_raca,
    cod_escolaridade,
    cod_classe_trabalho,
    cod_estado
    )

(
select enun.id_cliente,
       enun.genero,
       enun.idade,
       enun.qtde_filhos,
       enun.salario,
       enun.peso,
       enun.colesterol,
       eci.cod_estado_civil,
       rac.cod_raca,
       esc.cod_escolaridade,
       cod_classe_trabalho,
       est.cod_estado
       
      
from stg_enunciado as enun
inner join tb_classe_trabalho as ctr on ctr.classe_trabalho = enun.classe_trabalho 
inner join tb_estado as est on est.cod_estado = enun.id_estado 
inner join tb_estado_civil as eci on eci.estado_civil = enun.estado_civil
inner join tb_raca as rac on rac.raca = enun.raca
inner join tb_escolaridade as esc on esc.escolaridade = enun.escolaridade
)
'''

try:
    conn.execute(text(query))
    conn.commit()
    print('Dados inseridos com sucesso')

except Exception as e:
    print(f'Não foi possível inserir dados na tabela de Pessoa. Erro: {e}')

Dados inseridos com sucesso
